# Test

In [1]:
import os
import sys
import pandas as pd

In [2]:
sys.path.insert(0, os.path.abspath(".."))

In [3]:
from acro import ACRO

### Instantiate ACRO

In [4]:
acro = ACRO()

DEBUG:acro:path: /Users/j4-smith/GitHub/AI-SDC/ACRO/acro/default.yaml
DEBUG:acro:config: {'output_template_file': 'ACRO output template v01b.xlsm', 'safe_SDC_set': 'Default', 'safe_tests': 'threshold nk pratio maxmin', 'safe_threshold': 10, 'safe_dof_threshold': 10, 'safe_nk_n': 2, 'safe_nk_k': 0.9, 'safe_pratio_p': 0.1, 'safe_SDC_variations': 'CIS ESS', 'safe_thresholdCIS': 60, 'safe_nk_nCIS': 5, 'safe_nk_kCIS': 0.5, 'safe_testsCIS': 'threshold nk pratio', 'safe_thresholdESS': 15, 'safe_dof_thresholdESS': 10, 'safe_nk_nESS': 2, 'safe_nk_kESS': 0.9, 'safe_pratio_pESS': 0.15, 'safe_testsESS': 'nk pratio'}


### Load test data

In [5]:
path = os.path.join("../data", "test_data.dta")
df = pd.read_stata(path)
df.head()

,charity,grant_type,index,year,inc_activity,inc_grants,inc_donations,inc_other,inc_total,total_costs,...,sh_staff_grants_given,sh_assets_grants_given,sh_income_balance,sh_staff_balance,sh_assets_balance,sh_income_assets,sh_staff_assets,sh_income_staff_costs,sh_assets_staff_costs,wgt
0,4Children,R,1.0,2011,2880902.0,9603182.0,91404.0,310947.0,12886435.0,12127472.0,...,NaN,NaN,0.072636,0.135971,0.767809,0.094602,0.177090,0.534203,5.646843,1.0
1,4Children,R,1.0,2014,6810520.0,18768904.0,58002.0,401879.0,26039304.0,25493796.0,...,NaN,NaN,0.057641,0.089150,1.001396,0.057560,0.089026,0.646561,11.232729,1.0
2,4Children,R,1.0,2015,7199403.0,21638036.0,132191.0,512654.0,29482284.0,32290108.0,...,NaN,NaN,-0.049619,-0.079828,-0.620210,0.080004,0.128711,0.621583,7.769365,1.0
3,4Children,R,1.0,2013,5573013.0,15194731.0,228844.0,267156.0,21263744.0,20989048.0,...,NaN,NaN,0.045740,0.068251,1.008259,0.045365,0.067692,0.670166,14.772749,1.0
4,4Children,R,1.0,2010,2056816.0,7335103.0,110256.0,424628.0,9926803.0,9769816.0,...,NaN,NaN,0.057696,0.122532,0.567539,0.101660,0.215901,0.470862,4.631749,1.0


## Create a simple linear regression in three ways
 and print the results object returned and how to access the degrees of freedom
 
 I will start with inc_total vs inc_grants

### First sklearn


In [8]:
from sklearn.linear_model import LinearRegression

df.dropna(inplace=True)
y = df['inc_total'].values.reshape(-1, 1)  # values converts it into a numpy array
x = df['inc_grants'].values.reshape(-1, 1)  # -1 means that calculate the dimension of rows, but have 1 column
linear_regressor = LinearRegression()  # create object for the class
res= linear_regressor.fit(x, y)  # perform linear regression
y_pred = linear_regressor.predict(x)  # make predictions

In [34]:
print(f'intercept {linear_regressor.intercept_}, coeffecients {linear_regressor.coef_}')

intercept [29906768.], coeffecients [[1.8979713]]


### now a linear regression with two input variables

In [36]:
x2 = df[['inc_grants','inc_activity']].values.reshape(-1, 2)  # -1 means that calculate the dimension of rows, but have 1 column
bilinear_regressor = LinearRegression()  # create object for the class
res2= bilinear_regressor.fit(x2, y)  # perform linear regression
y_pred2 = bilinear_regressor.predict(x2)  # make predictions

print(f'intercept {bilinear_regressor.intercept_}, coeffecients {bilinear_regressor.coef_}')

intercept [12081892.], coeffecients [[1.723166  1.2810115]]


### for completeness a linear regression with three input variables but no intercept

In [41]:
x3 = df[['inc_grants','inc_activity','inc_donations']].values.reshape(-1, 3)  # -1 means that calculate the dimension of rows, but have 1 column
trilinear_regressor = LinearRegression(fit_intercept=False)  # create object for the class
res3= trilinear_regressor.fit(x3, y)  # perform linear regression
y_pred3 = trilinear_regressor.predict(x3)  # make predictions


print(f'intercept {trilinear_regressor.intercept_}, coeffecients {trilinear_regressor.coef_}')

intercept 0.0, coeffecients [[1.0145854 1.0113673 1.0529848]]


### Function to calculate the degrees of freedom of a model 
(number of parameters that can vary) will depend on whether an intercept is allowed.

In [17]:
type(linear_regressor)

sklearn.linear_model._base.LinearRegression

In [29]:
import sklearn
def get_dof(model)->int:
    if isinstance( model,sklearn.linear_model._base.LinearRegression):
        dof =  model.n_features_in_ 
        if model.fit_intercept :
            dof = dof +1
    return dof


In [59]:
print( 'Function reports that the Dof for:\n'
      f'\t\tsimple linear model is {get_dof(linear_regressor)}\n'
      f'\t\tmodel with two variables and intercept is {get_dof(bilinear_regressor)}\n'
      f'\t\tand for model with 3 variables and no intercept is {get_dof(trilinear_regressor)}.'
     )

Function reports that the Dof for:
		simple linear model is 2
		model with two variables and intercept is 3
		and for model with 3 variables and no intercept is 3.


## Polynomial regression using sklearn models by transforming data
### starting with a simple single input value

In [54]:
from sklearn.preprocessing import PolynomialFeatures

#get a clean version of the data
path = os.path.join("../data", "test_data.dta")
df = pd.read_stata(path)
df.dropna(inplace=True)
y = df['inc_total'].values.reshape(-1, 1)  # values converts it into a numpy array
x = df['inc_grants'].values.reshape(-1, 1)  # -1 means that calculate the dimension of rows, but have 1 column


#one liner to make new data rather than explicitly calling constructor,fit, then transform
degree= 2
x_ =  PolynomialFeatures(degree=degree, include_bias=False).fit_transform(x)

print(f'so to fit a polynomial of degree {degree}, data now has shape {x_.shape}')
print(x_[0:5,:])

#fit the data
polymodel = LinearRegression().fit(x_, y)
print(f'intercept {polymodel.intercept_}, coefficients {polymodel.coef_}')
print(f'and our function reports the degrees of freedom as {get_dof(polymodel)}')

so to fit a polynomial of degree 2, data now has shape (640, 2)
[[3.000e+04 9.000e+08]
 [0.000e+00 0.000e+00]
 [0.000e+00 0.000e+00]
 [0.000e+00 0.000e+00]
 [4.500e+04 2.025e+09]]
intercept [37651692.], coefficients [[4.0277270e-17 7.8147595e-09]]
and our function reports the degrees of freedom as 3


### and now multiple input values in the same framework
e.g. with 2 inputs and polynomial of degree 2, the estiamted regression function is 
𝑓(𝑥₁, 𝑥₂) = 𝑏₀ + 𝑏₁𝑥₁ + 𝑏₂𝑥₂ + 𝑏₃𝑥₁² + 𝑏₄𝑥₁𝑥₂ + 𝑏₅𝑥₂².

In [58]:
x2 = df[['inc_grants','inc_activity']].values.reshape(-1, 2)

degree= 2
x2_ =  PolynomialFeatures(degree=degree, include_bias=False).fit_transform(x2)

print(f'so to fit a polynomial of degree {degree}, data now has shape {x2_.shape}')
print('first five rows are:')
print(x2_[0:5,:])

#fit the data
polymodel2 = LinearRegression().fit(x2_, y)
print(f'intercept {polymodel2.intercept_}, coefficients {polymodel2.coef_}')
print(f'and our function reports the degrees of freedom as {get_dof(polymodel2)}')

so to fit a polynomial of degree 2, data now has shape (640, 5)
first five rows are:
[[3.0000000e+04 8.1703200e+05 9.0000000e+08 2.4510960e+10 6.6754131e+11]
 [0.0000000e+00 5.2915700e+05 0.0000000e+00 0.0000000e+00 2.8000711e+11]
 [0.0000000e+00 4.2903900e+05 0.0000000e+00 0.0000000e+00 1.8407447e+11]
 [0.0000000e+00 4.8266800e+05 0.0000000e+00 0.0000000e+00 2.3296840e+11]
 [4.5000000e+04 7.3457300e+05 2.0249999e+09 3.3055785e+10 5.3959750e+11]]
intercept [23210140.], coefficients [[ 4.8942300e-16 -4.0856207e-14  3.0109324e-09  6.4171914e-08
   1.9352635e-09]]
and our function reports the degrees of freedom as 6


### what if they provide intercept in data rather than in call to model

In [68]:
x3_ =  PolynomialFeatures(degree=degree).fit_transform(x2)

print(f'so to fit a polynomial of degree {degree}', 
      'but including an intercept,'
      f'data now has shape {x3_.shape}')
print('first five rows are:')
print(f'{x3_[0:5,:]:}')

#fit the data
polymodel3 = LinearRegression(fit_intercept=False).fit(x3_, y)
print(f'intercept {polymodel3.intercept_}, coefficients {polymodel3.coef_}')
print(f'and our function reports the degrees of freedom as {get_dof(polymodel3)}')

so to fit a polynomial of degree 2 but including an intercept,data now has shape (640, 6)
first five rows are:
[[1.0000000e+00 3.0000000e+04 8.1703200e+05 9.0000000e+08 2.4510960e+10
  6.6754131e+11]
 [1.0000000e+00 0.0000000e+00 5.2915700e+05 0.0000000e+00 0.0000000e+00
  2.8000711e+11]
 [1.0000000e+00 0.0000000e+00 4.2903900e+05 0.0000000e+00 0.0000000e+00
  1.8407447e+11]
 [1.0000000e+00 0.0000000e+00 4.8266800e+05 0.0000000e+00 0.0000000e+00
  2.3296840e+11]
 [1.0000000e+00 4.5000000e+04 7.3457300e+05 2.0249999e+09 3.3055785e+10
  5.3959750e+11]]
intercept 0.0, coefficients [[ 1.1876057e-23 -5.3290705e-15  8.9290241e-15  3.4045633e-09
   6.8190076e-08  2.0831212e-09]]
and our function reports the degrees of freedom as 6


In [71]:
print('if they include the intercept when transforming,'
      ' making a first column of 1s in the data as above,'
      ' but then also include the intercept in the model'
      ' we get this:'
     )

#fit the data

polymodel4 = LinearRegression(fit_intercept=True).fit(x3_, y)
print(f'intercept {polymodel4.intercept_}, coefficients {polymodel4.coef_}')
print(f'and our function reports the degrees of freedom as {get_dof(polymodel4)}')

if they include the intercept when transforming, making a first column of 1s in the data as above, but then also include the intercept in the model we get this:
intercept [23210152.], coefficients [[0.0000000e+00 1.0658141e-14 1.0699744e-14 3.0109324e-09 6.4171907e-08
  1.9352597e-09]]
and our function reports the degrees of freedom as 7


### <span style="color:red">Question: What should we do in this case?   
- We are going to release less with lower degrees of freedom.
- So maybe we should say if intercept ==0.00000 we do not count it?   
  i.e. 
    >         if model.fit_intercept :
    >              dof = dof +1
  becomes
    >         if model.fit_intercept and not np.isclose(model.intercept_ ,0.0):
    >               dof = dof +1
 - I've used two clauses in case the intee4cept is not always nicely initialsied to zero when unused 

### so the result is that the user can transform the data at will, 
### and our function still correctly determtines the DoF
### TO-DO check the other way they can provide intercepts
 - i.e. by setting include_intercept True (default) in Polynomial

# TO-DO
1. Repeat this process for different ways of getting a linear regression i.e. via scipy.stats
   - in some cases we may be able to get at the dof directly if the fit process returns a results object
   - or we could consider getting at it direcrtly from the combination of the number of colu,mns in x and whether an intercept is allowed
   - either way, build up the get_dof() function
   
2. Extend to some common other forms of regression

3. use this get_dof() function within the safe framework ricahrd has created for safe_table 